# Web search Agent with tavily

https://python.langchain.com/docs/modules/agents/quick_start



In [1]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
search = TavilySearchResults(max_results=1)

In [2]:
tools = [search]

In [3]:
search('año comienzo dictadura militar chilena')

[{'url': 'https://enciclopediadehistoria.com/dictadura-militar-en-chile/',
  'content': 'Régimen dictatorial establecido por el general chileno Augusto Pinochet Ugarte, en 1973 que se prolongó hasta 1990.  ¿Qué fue la dictadura militar en Chile?  democristiano Patricio Aylwin. Su asunción, el 11 de marzo de 1990, puso un fin a la dictadura militar en Chile.  golpe de Estado de 1973.La dictadura militar en Chile fue un régimen dictatorial encabezado por el general Augusto Pinochet Ugarte, que comenzó el 11 de septiembre de 1973 y finalizó el 11 de marzo de 1990. Ese período de 17 años se inició con un golpe de Estado que derrocó al gobierno constitucional del presidente socialista Salvador Allende, quien gobernaba desde 1970.'}]

In [4]:
from langchain_together import Together
llm = Together(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    temperature=0,
    max_tokens=256,
    top_k=1,
    together_api_key=os.getenv("TOGETHER_API_KEY")
)

In [23]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")

In [24]:
prompt[0].prompt.template = prompt[0].prompt.template[0:394] + "```\n\n\nFor example, if you want to use a tool to search for information, your $JSON_BLOB might look like this:\n\n```\n{{\n    'action': 'tavily_search_results_json', \n    'action_input': {{'query': 'Example search query'}}\n}}```" + prompt[0].prompt.template[394:]

In [25]:
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_structured_chat_agent
#agent = create_openai_tools_agent(llm,tools,prompt)
agent = create_structured_chat_agent(llm, tools, prompt)

In [26]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [28]:
agent_executor.invoke({"input": "Cual es el tiempo en Santiago de Chile ahora?"})



> Entering new AgentExecutor chain...

Thought: The user is asking about the current weather in Santiago, Chile. I can use the tavily_search_results_json tool to find this information.
Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": {
    "query": "current weather in Santiago, Chile"
  }
}
```
[{'url': 'https://crisis24.garda.com/alerts/2024/01/chile-high-temperatures-forecast-in-the-santiago-metropolitan-region-through-at-least-feb-2', 'content': "Find Out More Chile: High temperatures forecast in the Santiago Metropolitan Region through at least Feb. 2  Chilean Meteorological Directorate Learn more Get Unlimited Access Webinar Webinar Webinar Follow us on:  Officials issue red alert for extreme heat in Chile's Santiago Metropolitan Region until Feb. 2. Transport, utility  High temperatures will likely persist across much of Chile's Santiago Metropolitan Region through at least Feb. 2. The29 Jan 2024 | 02:40 AM UTC Chile: High temperatures forecast in the S

{'input': 'Cual es el tiempo en Santiago de Chile ahora?',
 'output': 'High temperatures are forecast in the Santiago Metropolitan Region through at least Feb. 2.'}